# Demonstrate ``TuneAxis``

In this example, we demonstrate the `apstools.plans.TuneAxis()` plan.  The `TuneAxis()` support may be used to align (a.k.a. *tune*) a signal against an axis.

We'll use a software-only (not connected to hardware) motor as a positioner.  Here, we prepare a signal that is a computation based on the value of our positioner.  The computed signal is a model of a realistic diffraction peak ([pseudo-Voigt](https://en.wikipedia.org/wiki/Voigt_profile), a mixture of a Gaussian and a Lorentzian) one might encounter in a powder diffraction scan.  The model peak is a pseudo-voigt function to which some noise has been added.  Random numbers are used to modify the ideal pseudo-voigt function so as to simulate a realistic signal.

For this demo, we do not need the databroker since we do not plan to review any of this data after collection.  We'll display the data during the scan using the *LiveTable()* code.

In [1]:
from apstools.devices import SynPseudoVoigt
from apstools.plans import TuneAxis
from bluesky import RunEngine
from bluesky import plans as bp
from bluesky.callbacks import LiveTable
import numpy as np
from ophyd import EpicsMotor

RE = RunEngine({})


Figure out which workstation we are running.  The *mint-vm* host has a different IOC prefix.

In [2]:
IOC = "gp:"

Connect to our motor *before* we create the simulated detector signal.

In [3]:
m1 = EpicsMotor(f"{IOC}m1", name="m1")
m1.wait_for_connection()

Define a starting position, we'll use this later in the demo.

In [4]:
m1.move(-1.5)
starting_position = m1.position

## Setup the simulated detector signal.  

Make a simple scan with a simulated motor and the `SynPseudoVoigt()` signal.

In [5]:
from ophyd.sim import motor

det = SynPseudoVoigt('det', motor, 'motor',
    center=0, eta=0.5, scale=1, sigma=1, bkg=0)

live_table = LiveTable(["motor", "det"])

RE(bp.scan([det], motor, -3, 3, 19), live_table)



+-----------+------------+------------+------------+
|   seq_num |       time |      motor |        det |
+-----------+------------+------------+------------+
|         1 | 18:41:26.2 |     -3.000 |      0.056 |
|         2 | 18:41:26.3 |     -2.667 |      0.076 |
|         3 | 18:41:26.3 |     -2.333 |      0.110 |
|         4 | 18:41:26.3 |     -2.000 |      0.168 |
|         5 | 18:41:26.3 |     -1.667 |      0.257 |
|         6 | 18:41:26.3 |     -1.333 |      0.386 |
|         7 | 18:41:26.3 |     -1.000 |      0.553 |
|         8 | 18:41:26.3 |     -0.667 |      0.747 |
|         9 | 18:41:26.3 |     -0.333 |      0.923 |
|        10 | 18:41:26.3 |      0.000 |      1.000 |
|        11 | 18:41:26.3 |      0.333 |      0.923 |
|        12 | 18:41:26.3 |      0.667 |      0.747 |
|        13 | 18:41:26.3 |      1.000 |      0.553 |
|        14 | 18:41:26.3 |      1.333 |      0.386 |
|        15 | 18:41:26.3 |      1.667 |      0.257 |
|        16 | 18:41:26.3 |      2.000 |     

('e6f7c881-da2c-48dc-8d0b-a34c59e14f0b',)

Make a new signal with randomized values so that we have something interesting to find with `TuneAxis()`.

In [6]:
spvoigt = SynPseudoVoigt(
    'spvoigt', m1, 'm1', 
    center=-1.5 + 0.4*np.random.uniform(), 
    eta=0.2 + 0.5*np.random.uniform(), 
    sigma=0.001 + 0.05*np.random.uniform(), 
    scale=1e5,
    bkg=0.01*np.random.uniform())

Reveal the actual values.  These are the answers we expect to discover.

In [7]:
print(f"{spvoigt.scale = }")
print(f"{spvoigt.center = }")
print(f"{spvoigt.sigma = }")
print(f"{spvoigt.eta = }")
print(f"{spvoigt.bkg = }")

spvoigt.scale = 100000.0
spvoigt.center = -1.1553241219392254
spvoigt.sigma = 0.04952451841481895
spvoigt.eta = 0.6177326684139124
spvoigt.bkg = 0.00028271101691823233


We will add the actual values as metadata to these scans.

In [8]:
md = dict(
    activity = "TuneAxis development and testing",
    peak_model = "pseudo Voigt",
    peak_scale = spvoigt.scale,
    peak_center = spvoigt.center,
    peak_sigma = spvoigt.sigma,
    peak_eta = spvoigt.eta,
    peak_bkg = spvoigt.bkg
    )

## Set up the tuner

Create a *TuneAxis()* object.  The *tuner* needs to know the positioner, what range to scan to find the peak, *and* it needs the name of the signal to be scanned (since the signal list may have more than one signal).

In [9]:
tuner = TuneAxis([spvoigt], m1, signal_name=spvoigt.name)
tuner.width = 2.5
tuner.step_factor = tuner.num/2.5

Reconfigure the *LiveTable* to also show the simulated detector signal.

In [10]:
live_table = LiveTable(["m1", "spvoigt"])

# Multi-pass tune

Execute multiple passes to refine the centroid determination.
Each subsequent pass will reduce the width of the next scan by ``step_factor``.

In [11]:
RE(tuner.multi_pass_tune(), live_table, md=md)



+-----------+------------+------------+------------+
|   seq_num |       time |         m1 |    spvoigt |
+-----------+------------+------------+------------+
|         1 | 18:41:46.5 |   -2.75000 |     87.793 |
|         2 | 18:41:47.0 |   -2.47225 |    115.509 |
|         3 | 18:41:47.5 |   -2.19450 |    168.255 |
|         4 | 18:41:48.0 |   -1.91675 |    288.498 |
|         5 | 18:41:48.5 |   -1.63900 |    669.190 |
|         6 | 18:41:49.0 |   -1.36100 |   3420.444 |
|         7 | 18:41:49.5 |   -1.08325 |  33097.710 |
|         8 | 18:41:50.1 |   -0.80550 |   1242.005 |
|         9 | 18:41:50.6 |   -0.52775 |    410.580 |
|        10 | 18:41:51.1 |   -0.25000 |    212.575 |
+-----------+------------+------------+------------+
generator TuneAxis.multi_pass_tune ['59738fc9'] (scan num: 2)


PeakStats
================ =================================
key              result                           
================ =================================
x                m1          

('59738fc9-1916-4c84-8676-7396f887e1e6',
 'c308977e-d9af-459a-bc50-ae60b5fdbab0',
 'd664623a-8a3f-41ee-b2c3-967125428838',
 '0fe9137d-7751-4cb3-9b60-30738139e60e')

Show the results from the multi-pass tuning.

In [12]:
print("final: ", tuner.center)
print("max", tuner.peaks.max)
print("min", tuner.peaks.min)
for stat in tuner.stats:
    print("--", stat.cen, stat.fwhm)
print(f"{m1.position=}  det={spvoigt.get()}")

final:  -1.1553219950893325
max (-1.1675, 95369.1956078187)
min (-1.0445, 13436.789922094267)
-- Signal(name='PeakStats_cen', parent='PeakStats', value=-1.0885316873365967, timestamp=1642725711.1095855) Signal(name='PeakStats_fwhm', parent='PeakStats', value=0.2983769532192523, timestamp=1642725711.1095936)
-- Signal(name='PeakStats_cen', parent='PeakStats', value=-1.1552031419819602, timestamp=1642725715.6256607) Signal(name='PeakStats_fwhm', parent='PeakStats', value=0.15124853842779928, timestamp=1642725715.6256957)
-- Signal(name='PeakStats_cen', parent='PeakStats', value=-1.15533945541456, timestamp=1642725718.4310818) Signal(name='PeakStats_fwhm', parent='PeakStats', value=0.11067466001505988, timestamp=1642725718.4310913)
-- Signal(name='PeakStats_cen', parent='PeakStats', value=-1.1553219950893325, timestamp=1642725721.140155) Signal(name='PeakStats_fwhm', parent='PeakStats', value=0.1001537087161457, timestamp=1642725721.1401901)
m1.position=-1.15525  det=13480.750786315055


Compare the final position (just printed) with the expected value shown a couple steps back.

## Single-pass tune

Repeat but with only one pass.  Reset the motor to the starting position and increase the number of steps by a factor of three.

In [13]:
m1.move(starting_position)
tuner.num *= 3
RE(tuner.tune(), live_table, md=md)



+-----------+------------+------------+------------+
|   seq_num |       time |         m1 |    spvoigt |
+-----------+------------+------------+------------+
|         1 | 18:42:03.7 |   -2.75000 |     87.793 |
|         2 | 18:42:04.0 |   -2.66375 |     94.787 |
|         3 | 18:42:04.3 |   -2.57750 |    103.089 |
|         4 | 18:42:04.6 |   -2.49150 |    113.017 |
|         5 | 18:42:04.9 |   -2.40525 |    125.097 |
|         6 | 18:42:05.2 |   -2.31900 |    139.955 |
|         7 | 18:42:05.5 |   -2.23275 |    158.513 |
|         8 | 18:42:05.8 |   -2.14650 |    182.107 |
|         9 | 18:42:06.1 |   -2.06025 |    212.737 |
|        10 | 18:42:06.4 |   -1.97425 |    253.366 |
|        11 | 18:42:06.7 |   -1.88800 |    309.227 |
|        12 | 18:42:07.0 |   -1.80175 |    388.735 |
|        13 | 18:42:07.3 |   -1.71550 |    507.354 |
|        14 | 18:42:07.6 |   -1.62925 |    695.544 |
|        15 | 18:42:07.9 |   -1.54300 |   1020.184 |
|        16 | 18:42:08.2 |   -1.45700 |   16

('a381e111-3216-4f44-ab83-27cea6de3409',)

Compare the single-pass scan with the previous multi-pass scan.  Each used the same number of points overall.  

The results are comparable but we already knew the position of the peak approximately.

In [14]:
print("final: ", tuner.center)
print("max", tuner.peaks.max)
print("min", tuner.peaks.min)
print("centroid", tuner.peaks.cen)
print("FWHM", tuner.peaks.fwhm)
print(f"{m1.position=}  det={spvoigt.get()}")

final:  -1.155317253761389
max (-1.19825, 61557.72427484274)
min (-2.75, 87.79310954973654)
centroid -1.155317253761389
FWHM 0.18716133951232927
m1.position=-1.15525  det=212.57529955844245
